<a href="https://colab.research.google.com/github/ingabLee/Book_GenerateAIForComptureVision/blob/main/GenerateAI_Vision_Chapter_2_3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install pytorch-model-summary

In [2]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import pytorch_model_summary

from google.colab import drive

drive.mount('/content/drive')

dirWork = '/content/drive/MyDrive/VisionForGenerateAI/Chapter2'
dirGANResult = dirWork + '/GANResult'

os.makedirs(dirGANResult, exist_ok=True)

Mounted at /content/drive


In [3]:
import os
import numpy as np
import math
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [4]:
# 에포크와 배치 사이즈 설정
epochs, batch_size = 100, 64

# 학습에 필요한 파라미터을 설정. Adam옵티마이저 hyper parameter B1,B2를 각 0.5,0.999로 설정
lr, b1, b2 = 2e-4, 0.5, 0.999

#GAN에 입력될 노이즈 차원크기 설정
latent_dim = 100

img_size = 28
channels = 1
#GAN이 생성할 이미지 차원크기를 미리 지정
img_shape = (channels, img_size, img_size)

if torch.cuda.is_available():
  print("Train on GPU \n CUA is aviable")
  cuda = True
else :
  print("Train on CPU \n CUA is not aviable")
  cuda = False

Train on GPU 
 CUA is aviable


In [7]:
# MNIST dataset저장할 경로 설정
dirMNIST = dirWork + "/data/mnist"
os.makedirs(dirMNIST, exist_ok=True)

# MNIST dataset로드하고 크기조절, 텐서화, 정규화등 전처리를 수행하며,
# 배치사이즈, 셔플여부을 설정하여 GAN학습을 위한 로더를 구성
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        dirMNIST, train=True, download=True,
        transform=transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5],[0.5])]),
    ),
    batch_size=batch_size,
    shuffle=True
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 527kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.71MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.9MB/s]


In [19]:
# 생성자 클래스를 정의
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()

    # 생성자를 구성하는 블록을 선형 레이어와 leakyReLU 활성화 함수 레이어를 통해 구성한다.
    def block(in_feat, out_feat, normalize=True):
      layers = [nn.Linear(in_feat, out_feat)]
      if normalize:
        layers.append(nn.BatchNorm1d(out_feat, 0.8))
      layers.append(nn.LeakyReLU(0.2, inplace=True))
      return layers

    # 선언된 블록과 추가 선형레이어, 그리고 Tanh활성화 함수로 생성자를 구성한다.
    self.model = nn.Sequential(
        *block(latent_dim, 128,normalize=False),
        *block(128, 256),
        *block(256, 512),
        *block(512, 1024),
        nn.Linear(1024, int(np.prod(img_shape))),
        nn.Tanh()
    )

  def forward(self, z):
      # 모델 동작단계에서 입력 노이즈에 대한 생성자 출력을 지정한다.
      img = self.model(z)
      # 출력된 결과의 사이즈를 먼저 선언해둔 이미지 형태로 조절한다.
      img = img.view(img.shape[0], *img_shape)
      return img

In [20]:
# 판별자 클래스를 선언한다.
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    # 선형 레이어와 leakyReLU활성화 함수 레이어, 그리고 [0,1] 범위이 판별값 출력을
    # 위한 sigmoid활성화 함수 레이어로 판별자를 구성한다.
    self.model = nn.Sequential(
        nn.Linear(int(np.prod(img_shape)), 512),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(512, 256),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(256, 1),
        nn.Sigmoid(),
    )

  def forward(self, img):
    # 입력 이미지를 1차원으로 평탄화 한다.
    flat_img = img.view(img.size(0), -1)
    # 판별자가 입력에 대한 판별값을 출력한다. 판별값은 [0, 1] 범위를 갖는다.
    pred = self.model(flat_img)
    return pred

In [21]:
# 생성자와 판별자, 학습에 사용할 이진 교차 엔트로피 함수를 선언.
G=Generator()
D=Discriminator()
adversarial_loss = torch.nn.BCELoss()

# GPU가 활용가능할 경우 각 모델과 손실함수를 GPU에 로드한다.
if cuda:
  G.cuda()
  D.cuda()
  adversarial_loss.cuda()

In [25]:
# 모델의 최적화를 수행할 adam 옵티마이저를 정의한다.
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
# 생성 결과 이미지를 저장할 경로에 폴더를 생성
os.makedirs(dirGANResult, exist_ok=True)

# 에포크만큼 학습. epoch는 모델 및 데이터에 따라 조절할 수 있다.
for epoch in range(epochs):

  for i, (imgs, _) in enumerate(dataloader):
    # 진자 이미지 라벨값 1, 그리고 가짜 이미지 라벨값 0을 텐서형태로 각각
    # 배치사이즈 만큼 생성한다.
    real = Variable(Tensor(imgs.size(0), 1).fill_(1.0),
                    requires_grad=False)
    fake = Variable(Tensor(imgs.size(0),1).fill_(0.0),
                    requires_grad=False)
    real_imgs = Variable(imgs.type(Tensor))

    ## train Generator ##
    # 생성자 학습을 진행한다.랜덤 노이즈로부터 생성자가 만든 이미지를 판별한 값과
    # 1 사이의 이진교차 엔트로피 손실을 계산한다. 그리고 그래디언트를 전달하여
    #생성자를 업데이트 한다.
    optimizer_G.zero_grad()
    z = Variable(Tensor(np.random.normal(0, 1,
                                         (imgs.shape[0], latent_dim))))
    gen_imgs = G(z)
    g_loss = adversarial_loss(D(gen_imgs), real)
    g_loss.backward()
    optimizer_G.step()

    ## train discriminator ##
    # 판별자 학습을 진행한다. 각각 생성자가 만든 가짜 이미지이 판별값은 0,
    # 진짜 이미지 판별값은 1과 이진교차 엔트로피 손실을 계산하고
    # 그래디언트를 전달하여 판별자를 업데이트 한다.
    optimizer_D.zero_grad()
    real_loss = adversarial_loss(D(real_imgs), real)
    fake_loss = adversarial_loss(D(gen_imgs.detach()), fake)
    d_loss = (real_loss + fake_loss) / 2
    d_loss.backward()
    optimizer_D.step()

  # 한 에포크가 끝날때마다생성자가 만든 이미지를 저장한다.
  print("[Epoch%d/%d] [D loss:%f] [G loss:%f]" %
        ((epoch + 1), epochs, d_loss.item(), g_loss.item()))
  save_image(gen_imgs.data[:25], dirGANResult+"epoch_%d.png" % epoch, nrow=5, normalize=True)

# 학습이 종료된 후에 업데이트된 모델을 저장한다.
torch.save(G.state_dict(), dirGANResult+"/Generator.pth")
torch.save(D.state_dict(), dirGANResult+"/Discriminator.pth")

[Epoch1/100] [D loss:0.451876] [G loss:0.617509]
[Epoch2/100] [D loss:0.218859] [G loss:1.915246]
[Epoch3/100] [D loss:0.214227] [G loss:2.092120]
[Epoch4/100] [D loss:0.208579] [G loss:2.067677]
[Epoch5/100] [D loss:0.400720] [G loss:0.774979]
[Epoch6/100] [D loss:0.153706] [G loss:2.385167]
[Epoch7/100] [D loss:0.153958] [G loss:2.555676]
[Epoch8/100] [D loss:0.282926] [G loss:1.446465]
[Epoch9/100] [D loss:0.241898] [G loss:3.448091]
[Epoch10/100] [D loss:0.314380] [G loss:3.836847]
[Epoch11/100] [D loss:0.292057] [G loss:1.117122]
[Epoch12/100] [D loss:0.122918] [G loss:2.948808]
[Epoch13/100] [D loss:0.275141] [G loss:5.734448]
[Epoch14/100] [D loss:0.098829] [G loss:2.712463]
[Epoch15/100] [D loss:0.101055] [G loss:3.085324]
[Epoch16/100] [D loss:0.196645] [G loss:1.496794]
[Epoch17/100] [D loss:0.117629] [G loss:2.078698]
[Epoch18/100] [D loss:0.062403] [G loss:2.635151]
[Epoch19/100] [D loss:0.069634] [G loss:3.275043]
[Epoch20/100] [D loss:0.160250] [G loss:4.985387]
[Epoch21/

KeyboardInterrupt: 